## 2.2 - ReAct Prompting

## Preparation

In [1]:
#!pip install -q langchain==0.1.9 openai==1.13.3
!pip install -q langchain-openai langchain
#!pip uninstall langchain openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.0.65 requires langchain-community<0.3.0,>=0.2.16, but you have langchain-community 0.3.5 which is incompatible.
langchain-community 0.3.5 requires langchain<0.4.0,>=0.3.6, but you have langchain 0.2.17 which is incompatible.
langchain-community 0.3.5 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.2.43 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Imports
import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
#from langchain_community.llms import VLLMOpenAI
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [3]:
INFERENCE_SERVER_URL = "https://mistral-7b-instruct-v0-3-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443"
MODEL_NAME = "mistral-7b-instruct"
API_KEY= os.getenv('API_KEY')

#### Create the LLM instance

In [4]:
# LLM definition
llm = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base= f"{INFERENCE_SERVER_URL}/v1",
    model_name=MODEL_NAME,
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

#### Create the Prompt

In [15]:
template="""<s>[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe.
You will be asked a question, to which you must give an answer.
Your answer should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct but in a simple way and very concise
If you don't know the answer to a question, answer "I don't know".
<</SYS>>

### QUESTION:
{input}

### ANSWER:
[/INST]
"""
PROMPT_LLM = PromptTemplate(input_variables=["input"], template=template)

## First Query to LLMs

#### Create the Chain using the different components

In [16]:
conversation = LLMChain(llm=llm,
                            prompt=PROMPT_LLM,
                            verbose=False,
                            )

#### Let's talk...

In [17]:
first_input = "I received a damaged product in my recent order. What should I do to get a replacement?"
conversation.predict(input=first_input);

 I'm sorry to hear about the issue with your order. To get a replacement for a damaged product, you should contact the seller or the customer service of the company from which you purchased the item. They will likely ask for photos of the damage and may require the original order number or receipt. Once you provide them with this information, they should be able to help you with a replacement or a refund. If you're unsure how to reach out to them, check their website or contact details provided in your order confirmation email.

#### LLM with Chain of Thought in the Prompt

In [24]:
template="""<s>[INST]<<SYS>>
You are a helpful, respectful, and honest assistant. Always be as helpful as possible while being safe.
You will be asked a question, to which you must give an answer.
Your answer should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
When answering, think step-by-step and explain your reasoning before providing the final answer.
Explain it with bullet points if it's needed but using reasoning and step-by-step.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something incorrect.
If you don't know the answer to a question, answer "I don't know".
<</SYS>>

### QUESTION:
{input}

### ANSWER:
[/INST]
"""
PROMPT_COT = PromptTemplate(input_variables=["input"], template=template)

##### Chain of Thought Prompt
* When answering, think step-by-step and explain your reasoning before providing the final answer.
* Explain it with bullet points if it's needed but using reasoning and step-by-step.

In [27]:
conversation_cot = LLMChain(llm=llm,
                            prompt=PROMPT_COT,
                            verbose=False,
                            )

In [28]:
first_input = "I received a damaged product in my recent order. What should I do to get a replacement?"
conversation_cot.predict(input=first_input);

 To get a replacement for a damaged product in your recent order, follow these steps:

1. Document the damage: Take clear and detailed photos of the damaged product from different angles. This will serve as evidence when you contact the seller or customer service.

2. Contact the seller or customer service: Reach out to the seller or the platform's customer service (e.g., Amazon, eBay) through their official channels, such as email, phone, or chat support. Provide them with the order number, a brief description of the issue, and attach the photos of the damaged product.

3. Explain the problem: Clearly explain the issue with the product and why it is damaged. Be polite and professional in your communication.

4. Request a replacement: Ask the seller or customer service representative if they can provide a replacement for the damaged product. If they agree, ask about the process for receiving the replacement, including shipping costs, estimated delivery time, and return instructions for

## What's Next?

Let's head over to give our LLM Conversational Memory!